In [59]:
import os
import numpy as np
import pandas as pd
import altair as alt
import sys
import json

sys.path.insert(0, '..')
from agents import PsAgent
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [60]:
lista = [i for i in os.listdir('../data/models/colision_001') if i not in ['.ipynb_checkpoints']]

In [61]:
lista

['2025-03-26_12-42-15.577204__17',
 '2025-03-26_12-42-15.569653__10',
 '2025-03-26_12-42-15.569013__9',
 '2025-03-26_12-42-15.570511__12',
 '2025-03-26_12-42-15.614822__24',
 '2025-03-26_12-42-15.626970__26',
 '2025-03-26_12-42-15.659025__31',
 '2025-03-26_12-42-15.578077__19',
 '2025-03-26_12-42-15.606855__23',
 '2025-03-26_12-42-15.572245__15',
 '2025-03-26_12-42-15.638916__27',
 '2025-03-26_12-42-15.572763__16',
 '2025-03-26_12-42-15.566782__5',
 '2025-03-26_12-42-15.562115__2',
 '2025-03-26_12-42-15.586778__20',
 '2025-03-26_12-42-15.567807__7',
 '2025-03-26_12-42-15.561639__1',
 '2025-03-26_12-42-15.570988__13',
 '2025-03-26_12-42-15.646750__28',
 '2025-03-26_12-42-15.569997__11',
 '2025-03-26_12-42-15.654794__29',
 '2025-03-26_12-42-15.563072__4',
 '2025-03-26_12-42-15.595375__22',
 '2025-03-26_12-42-15.577614__18',
 '2025-03-26_12-42-15.615524__25',
 '2025-03-26_12-42-15.568455__8',
 '2025-03-26_12-42-15.571659__14',
 '2025-03-26_12-42-15.562591__3',
 '2025-03-26_12-42-15.567251

In [62]:
dados = []
for model in lista:
    path = f'../data/models/colision_001/{model}'
    
    with open(f'{path}/args.json', 'r') as file:
        args = json.load(file)
        
    n_reflect = args['num_reflections']
    tao = args['tao']
    colision_flag = args['colision']
    agent = PsAgent.load(path)
    matrix = agent.h_matrix
    if colision_flag:
        for state in [0,1]:
            for colision in [0,1]:
                for T in range(0,300):
                    observation = [state,T,colision]
                    percept = agent.percept_preprocess(observation)

                    h_vector = matrix[:, percept]
                    sum_ = np.sum(h_vector)
                    probs = h_vector/sum_

                    nao_troca = probs[0]
                    troca = probs[1]

                    _ = {
                        'model':model,
                        'reflections':args['num_reflections'],
                        'T':T/tao,
                        'state':state,
                        'colision_flag':colision_flag,
                        'colision':colision,
                        'troca':troca,
                        'nao_troca':nao_troca,
                    }

                    dados.append(_)
    else:
        for state in [0,1]:
            for T in range(0,300):
                observation = [state,T]
                percept = agent.percept_preprocess(observation)

                h_vector = matrix[:, percept]
                sum_ = np.sum(h_vector)
                probs = h_vector/sum_

                nao_troca = probs[0]
                troca = probs[1]

                _ = {
                    'model':model,
                    'reflections':args['num_reflections'],
                    'T':T/tao,
                    'state':state,
                    'colision_flag':colision_flag,
                    'colision':np.nan,
                    'troca':troca,
                    'nao_troca':nao_troca,
                }

                dados.append(_)

# Sem colisão

In [63]:
db = pd.DataFrame(dados)
db = db[~db['colision_flag'].apply(bool)]
db = db.sort_values(['model','state','T']).reset_index(drop = True)

db['prod_nao_troca'] = db.groupby(['model','state'])['nao_troca'].cumprod().shift(1)
db['P_T_troca'] = db['prod_nao_troca']*db['troca']
db['sum_P_T_troca'] = db.groupby(['model','state'])['P_T_troca'].cumsum().shift(1)

db_mean = db[db['T'].between(2,200)].groupby(['reflections','T','state'])[
    ['troca', 'P_T_troca', 'sum_P_T_troca']
].mean().reset_index()

db_mean.head()

,reflections,T,state,troca,P_T_troca,sum_P_T_troca


In [64]:
alt.Chart(db_mean).mark_line().encode(
    x = 'T',
    y = alt.Y('troca').axis(format = '%'),
    row = 'state:N',
    color = 'reflections:N'
).properties(height = 100).resolve_scale(y = 'independent')

alt.Chart(...)

In [65]:
alt.Chart(db_mean).mark_line().encode(
    x = 'T',
    y = alt.Y('P_T_troca').axis(format = '%'),
    row = 'state:N',
    color = 'reflections:N'
).properties(height = 100).resolve_scale(y = 'independent')

alt.Chart(...)

In [66]:
alt.Chart(db_mean).mark_line().encode(
    x = 'T',
    y = alt.Y('sum_P_T_troca').axis(format = '%'),
    color = 'state:N',
).properties(height = 100).resolve_scale(y = 'independent')

alt.Chart(...)

# Com colisão

In [67]:
db = pd.DataFrame(dados)
db = db[db['colision_flag'].apply(bool)]
db = db.sort_values(['model','state','colision','T']).reset_index(drop = True)

db['prod_nao_troca'] = db.groupby(['model','state','colision'])['nao_troca'].cumprod().shift(1)
db['P_T_troca'] = db['prod_nao_troca']*db['troca']
db['sum_P_T_troca'] = db.groupby(['model','state','colision'])['P_T_troca'].cumsum().shift(1)

db_mean = db[db['T'] > 2/tao].groupby(['reflections','T','state','colision'])[
    ['troca', 'P_T_troca', 'sum_P_T_troca']
].mean().reset_index()

db_mean.head()

,reflections,T,state,colision,troca,P_T_troca,sum_P_T_troca
0,0,0.0003,0,0,0.001167,0.001162,0.003458
1,0,0.0003,0,1,0.002908,0.002882,0.007475
2,0,0.0003,1,0,0.000067,0.000067,0.001505
3,0,0.0003,1,1,0.000849,0.000848,0.002612
4,0,0.0004,0,0,0.001428,0.001421,0.004620


In [68]:
alt.Chart(db_mean).mark_line().encode(
    x = 'T',
    y = alt.Y('troca').axis(format = '%'),
    row = 'state:N',
    column = 'colision:N',
    color = 'reflections:N'
).properties(height = 100).resolve_scale(y = 'independent')

alt.Chart(...)

In [69]:
alt.Chart(db_mean).mark_line().encode(
    x = 'T',
    y = alt.Y('P_T_troca').axis(format = '%'),
    row = 'state:N',
    column = 'colision:N',
    color = 'reflections:N'
).properties(height = 100).resolve_scale(y = 'independent')

alt.Chart(...)

In [70]:
alt.Chart(db_mean).mark_line().encode(
    x = 'T',
    y = alt.Y('sum_P_T_troca').axis(format = '%'),
    color = 'state:N',
    row = 'colision:N',
).properties(height = 100).resolve_scale(y = 'independent')

alt.Chart(...)

In [71]:
 alt.Chart(db_mean).mark_line().encode(
    x = 'T',
    y = alt.Y('sum_P_T_troca').axis(format = '%'),
    row = 'state:N',
    color = 'colision:N',
).properties(height = 100).resolve_scale(y = 'independent')

alt.Chart(...)